#### 포디블록 구조 추출 AI 경진대회
---
출처: https://dacon.io/competitions/official/236046/overview/description

In [14]:
import tensorflow as tf
import pandas as pd
import random
import cv2
from tqdm import tqdm
from os.path import exists
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB3, preprocess_input

In [3]:
IMG_SIZE = 224
EPOCHS = 10
BATCH_SIZE = 128

1. 데이터 로딩

In [4]:
df = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [5]:
df.head()

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0


2. 학습/검증 데이터 분리

In [17]:
df = df.sample(frac=1)
train_len = int(len(df) * 0.8)

train = df[:train_len].reset_index(drop=True)
val = df[train_len:].reset_index(drop=True)

In [18]:
path_train = train['img_path']
path_valid = val['img_path']
path_test = test['img_path']

3. TFRecord 생성

---
TFRecord 안은 Protocol Buffer이라는 바이너리 포맷으로 되어 있다. 한 번 TFRecord를 작성하는 것으로 데이터의 생성, 가공 비용을 줄일 수 있는 경우가 있다.

In [13]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [19]:
## TFRecord writer 생성
writer_image = tf.io.TFRecordWriter('image_train.tfr')
for i_, path_ in tqdm(enumerate(path_train)):

    src = cv2.imread(path_)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    dst = cv2.resize(dst, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    bimage = dst.tobytes()
    
    classes = np.array(train.loc[i_, 'A':'J'], dtype=np.uint8).tobytes()
    
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': _bytes_feature(bimage),
                'classes': _bytes_feature(classes)
            }
            )
        )
    
    writer_image.write(example.SerializeToString())
    
writer_image.close()

26395it [02:53, 152.53it/s]


In [20]:
writer_image_valid = tf.io.TFRecordWriter('image_valid.tfr')
for i_, path_ in tqdm(enumerate(path_valid)):

    src = cv2.imread(path_)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    dst = cv2.resize(dst, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)    
    bimage = dst.tobytes()
    
    classes = np.array(val.loc[i_, 'A':'J'], dtype=np.uint8).tobytes()
    
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': _bytes_feature(bimage),
                'classes': _bytes_feature(classes)            
            }
            )
        )
    
    writer_image_valid.write(example.SerializeToString())
    
writer_image_valid.close()

6599it [00:44, 149.01it/s]


In [22]:
writer_image_test = tf.io.TFRecordWriter('image_test.tfr')

for i_, path_ in tqdm(enumerate(path_test)):

    src = cv2.imread(path_)
    dst = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    dst = cv2.resize(dst, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    
    bimage = dst.tobytes()
    
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': _bytes_feature(bimage)
            }
            )
        )
    
    writer_image_test.write(example.SerializeToString())
    
writer_image_test.close()

1460it [00:10, 134.40it/s]


In [23]:
## tfrecord file을 data로 parsing해주는 function
def _parse_function(tfrecord_serialized):
    features={'image': tf.io.FixedLenFeature([], tf.string),
              'classes': tf.io.FixedLenFeature([], tf.string)
             }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [IMG_SIZE, IMG_SIZE, 3])
#     image = tf.cast(image, tf.float32)/255. 

    classes = tf.io.decode_raw(parsed_features['classes'], tf.uint8)    
    classes = tf.squeeze(classes)

    return image, classes

def _parse_function2(tfrecord_serialized):
    features={'image': tf.io.FixedLenFeature([], tf.string)             }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [IMG_SIZE, IMG_SIZE, 3])
#     image = tf.cast(image, tf.float32)/255. 

    return image

In [24]:
## train dataset 만들기
train_dataset = tf.data.TFRecordDataset('image_train.tfr')
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(2000).prefetch(tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE)

In [25]:
## validation dataset 만들기
val_dataset = tf.data.TFRecordDataset('image_valid.tfr')
val_dataset = val_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE)

5. 모델 생성